<a href="https://colab.research.google.com/github/dgiri1/Test_Data/blob/master/Traffic_Signals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import File from the local file system to Google Colab


In [41]:
from google.colab import files

uploaded = files.upload()

#for fn in uploaded.keys():
  #print('User uploaded file "{name}" with length {length} bytes'.format(
      #name=fn, length=len(uploaded[fn])))

Saving Signal Systems and Signals.xlsx to Signal Systems and Signals (2).xlsx


In [0]:
import numpy as np
import pandas as pd

Read the excel file into the pandas dataframe

In [43]:
df = pd.read_excel('OLD Signal IDs_updated.xlsx',sheet_name='OLD Signal IDs')

df.head()

,Master Asset,System Master,System Name,Division,Travel Time Flags,Travel Time Inventoried Miles,AADT Inventoried Miles,Maximum Segment AADT,Annual VMT (Millions),Safety Inventoried Miles,Combined Safety Score
0,MA# 10102,01-0067,"MA# 10102, US 13 - NC 42-561 (Academy St), Div...",Division 1,0.0,4.59,2.10,16000.0,9.14,2.34,79.0
1,MA# 10103,01-0039,"MA# 10103, US 13 - NC 42-561 (Memorial Dr), Di...",Division 1,0.0,2.63,1.58,21000.0,8.71,1.82,66.5
2,MA# 10106,01-0316,"MA# 10106, US 158 (Croatan Hwy) North - NC 12,...",Division 1,1.0,5.51,6.83,28000.0,34.32,6.86,56.5
3,MA# 10107,01-0260,"MA# 10107, US 158 (Croatan Hwy) South, Divisio...",Division 1,0.0,60.64,25.35,39000.0,175.55,24.93,49.6
4,MA# 10108,01-0014,"MA# 10108, US 17- US 17 Bus (Elizabeth City), ...",Division 1,3.0,20.17,11.68,26000.0,65.26,11.04,66.0


Extracted only the numercial values from the Master Asset column

In [0]:
df['Master Asset'] = df['Master Asset'].apply(lambda x: x.split('# ')[1])

Renamed the columns

In [45]:
df = df.rename(columns={'Master Asset':'Old Signal MA', 'System Master':'Old Signal ID', 'System Name':'Old Signal Name'});
name = df.columns[:3]
df1 = df[name]
df1.head()

,Old Signal MA,Old Signal ID,Old Signal Name
0,10102,01-0067,"MA# 10102, US 13 - NC 42-561 (Academy St), Div..."
1,10103,01-0039,"MA# 10103, US 13 - NC 42-561 (Memorial Dr), Di..."
2,10106,01-0316,"MA# 10106, US 158 (Croatan Hwy) North - NC 12,..."
3,10107,01-0260,"MA# 10107, US 158 (Croatan Hwy) South, Divisio..."
4,10108,01-0014,"MA# 10108, US 17- US 17 Bus (Elizabeth City), ..."


In [46]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Signal Systems and Signals.xlsx to Signal Systems and Signals (3).xlsx
User uploaded file "Signal Systems and Signals.xlsx" with length 179957 bytes


In [47]:
df2 = pd.read_excel('Signal Systems and Signals (3).xlsx',sheet_name='Master')

df2.head()

,Signal Inventory Number,Signal Type,Signal System Number,System Description
0,01-0067,Master,10102,US 13 - NC 42-561 (Academy St)
1,01-0039,Master,10103,US 13 - NC 42-561 (Memorial Dr)
2,01-0316,Master,10106,US 158 (Croatan Hwy) North - NC 12
3,01-0260,Master,10107,US 158 (Croatan Hwy) South
4,01-0014,Master,10108,US 17-US 17 Bus (Elizabeth City)


In [48]:
df2 = df2.rename(columns={'Signal Inventory Number':'New Signal ID'})
df2.head()


,New Signal ID,Signal Type,Signal System Number,System Description
0,01-0067,Master,10102,US 13 - NC 42-561 (Academy St)
1,01-0039,Master,10103,US 13 - NC 42-561 (Memorial Dr)
2,01-0316,Master,10106,US 158 (Croatan Hwy) North - NC 12
3,01-0260,Master,10107,US 158 (Croatan Hwy) South
4,01-0014,Master,10108,US 17-US 17 Bus (Elizabeth City)


In [49]:
df2 = df2.rename(columns={'System Description':'New Signal Name','Signal System Number':'New Signal MA'})
df2.head()

,New Signal ID,Signal Type,New Signal MA,New Signal Name
0,01-0067,Master,10102,US 13 - NC 42-561 (Academy St)
1,01-0039,Master,10103,US 13 - NC 42-561 (Memorial Dr)
2,01-0316,Master,10106,US 158 (Croatan Hwy) North - NC 12
3,01-0260,Master,10107,US 158 (Croatan Hwy) South
4,01-0014,Master,10108,US 17-US 17 Bus (Elizabeth City)


In [50]:
df2 = df2[['New Signal MA','New Signal ID','New Signal Name']]
df2['New Signal MA'] = df2['New Signal MA'].astype(str).apply(lambda x: x.split('-')[0])
df2.head()

,New Signal MA,New Signal ID,New Signal Name
0,10102,01-0067,US 13 - NC 42-561 (Academy St)
1,10103,01-0039,US 13 - NC 42-561 (Memorial Dr)
2,10106,01-0316,US 158 (Croatan Hwy) North - NC 12
3,10107,01-0260,US 158 (Croatan Hwy) South
4,10108,01-0014,US 17-US 17 Bus (Elizabeth City)


Merged with the dataframe with the missing MA values by matching the Signal Name

In [51]:
df3 = pd.read_excel('OLD Signal IDs_updated.xlsx',sheet_name='Missing MA values', skiprows=4)

df3['System Number'] = df3['System Number'].astype(str).apply(lambda x: x.split('-')[0])
df3.head()

temp = df1.merge(df3,how='left',left_on='Old Signal Name',right_on='System Name')
temp.head()


,Old Signal MA,Old Signal ID,Old Signal Name,Master Asset,System Master,System Name,Division-SID in new table,System Number
0,10102,01-0067,"MA# 10102, US 13 - NC 42-561 (Academy St), Div...",NaN,NaN,NaN,NaN,NaN
1,10103,01-0039,"MA# 10103, US 13 - NC 42-561 (Memorial Dr), Di...",NaN,NaN,NaN,NaN,NaN
2,10106,01-0316,"MA# 10106, US 158 (Croatan Hwy) North - NC 12,...",NaN,NaN,NaN,NaN,NaN
3,10107,01-0260,"MA# 10107, US 158 (Croatan Hwy) South, Divisio...",NaN,NaN,NaN,NaN,NaN
4,10108,01-0014,"MA# 10108, US 17- US 17 Bus (Elizabeth City), ...",NaN,NaN,NaN,NaN,NaN


Function used to replace the XXXXX  values with their corresponding MA number

In [0]:

def replace_xxx(df):
  if df['Old Signal MA'] == 'XXXXX':
    return df['System Number']
  else:
    return df['Old Signal MA']
    


In [53]:
df1['Old Signal MA'] = temp.apply(replace_xxx,axis = 1)
df1['Old Signal MA'].size


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


390

Merged the two tables by matching the MA numbers

In [54]:
final = df1.merge(df2,how='outer',left_on='Old Signal MA', right_on='New Signal MA');
final.head()

,Old Signal MA,Old Signal ID,Old Signal Name,New Signal MA,New Signal ID,New Signal Name
0,10102,01-0067,"MA# 10102, US 13 - NC 42-561 (Academy St), Div...",10102,01-0067,US 13 - NC 42-561 (Academy St)
1,10103,01-0039,"MA# 10103, US 13 - NC 42-561 (Memorial Dr), Di...",10103,01-0039,US 13 - NC 42-561 (Memorial Dr)
2,10106,01-0316,"MA# 10106, US 158 (Croatan Hwy) North - NC 12,...",10106,01-0316,US 158 (Croatan Hwy) North - NC 12
3,10107,01-0260,"MA# 10107, US 158 (Croatan Hwy) South, Divisio...",10107,01-0260,US 158 (Croatan Hwy) South
4,10108,01-0014,"MA# 10108, US 17- US 17 Bus (Elizabeth City), ...",10108,01-0014,US 17-US 17 Bus (Elizabeth City)


In [55]:
from google.colab import files

uploaded = files.upload()

old_count = pd.read_excel('OLD Signal Number Lookup.xlsx',sheet_name='Sheet2', skiprows=3)
old_count.head()


Saving OLD Signal Number Lookup.xlsx to OLD Signal Number Lookup (1).xlsx


,Row Labels,Total
0,,2.0
1,<Null>,56.0
2,01-0004,4.0
3,01-0014,25.0
4,01-0039,4.0


Matched the number of old signals to the table by joining on the Signal IDs

In [56]:
temp2 = final.merge(old_count, how='left',left_on='Old Signal ID',right_on='Row Labels')

temp2.head()



,Old Signal MA,Old Signal ID,Old Signal Name,New Signal MA,New Signal ID,New Signal Name,Row Labels,Total
0,10102,01-0067,"MA# 10102, US 13 - NC 42-561 (Academy St), Div...",10102,01-0067,US 13 - NC 42-561 (Academy St),01-0067,5.0
1,10103,01-0039,"MA# 10103, US 13 - NC 42-561 (Memorial Dr), Di...",10103,01-0039,US 13 - NC 42-561 (Memorial Dr),01-0039,4.0
2,10106,01-0316,"MA# 10106, US 158 (Croatan Hwy) North - NC 12,...",10106,01-0316,US 158 (Croatan Hwy) North - NC 12,01-0316,10.0
3,10107,01-0260,"MA# 10107, US 158 (Croatan Hwy) South, Divisio...",10107,01-0260,US 158 (Croatan Hwy) South,01-0260,25.0
4,10108,01-0014,"MA# 10108, US 17- US 17 Bus (Elizabeth City), ...",10108,01-0014,US 17-US 17 Bus (Elizabeth City),01-0014,25.0


In [0]:
temp2 = temp2.rename(columns={'Total':'Old Signal Count'})
temp2 = temp2.drop(columns='Row Labels')


In [59]:

new_count = pd.read_excel('Signal Systems and Signals (3).xlsx',sheet_name='Sheet1', skiprows=3)
new_count.head()


,Row Labels,Total
0,10102,5
1,10103,4
2,10106,10
3,10107,25
4,10108,24


In [60]:
new_count['Row Labels'] = new_count['Row Labels'].astype(str).apply(lambda x: x.split('-')[0])
new_count = new_count.rename(columns={'Total':'New Signal Count'})
new_count.head()

,Row Labels,New Signal Count
0,10102,5
1,10103,4
2,10106,10
3,10107,25
4,10108,24


In [61]:
temp3 = temp2.merge(new_count, how='left',left_on='New Signal MA',right_on='Row Labels')
temp3 = temp3.drop(columns='Row Labels')

temp3.head()

,Old Signal MA,Old Signal ID,Old Signal Name,New Signal MA,New Signal ID,New Signal Name,Old Signal Count,New Signal Count
0,10102,01-0067,"MA# 10102, US 13 - NC 42-561 (Academy St), Div...",10102,01-0067,US 13 - NC 42-561 (Academy St),5.0,5.0
1,10103,01-0039,"MA# 10103, US 13 - NC 42-561 (Memorial Dr), Di...",10103,01-0039,US 13 - NC 42-561 (Memorial Dr),4.0,4.0
2,10106,01-0316,"MA# 10106, US 158 (Croatan Hwy) North - NC 12,...",10106,01-0316,US 158 (Croatan Hwy) North - NC 12,10.0,10.0
3,10107,01-0260,"MA# 10107, US 158 (Croatan Hwy) South, Divisio...",10107,01-0260,US 158 (Croatan Hwy) South,25.0,25.0
4,10108,01-0014,"MA# 10108, US 17- US 17 Bus (Elizabeth City), ...",10108,01-0014,US 17-US 17 Bus (Elizabeth City),25.0,24.0


Downloaded the dataframe from Colab as an excel file to the local file system.

In [0]:
temp3.to_excel('final_updated_new.xlsx',sheet_name='Sheet')



files.download('final_updated_new.xlsx')

In [0]:
df1['Old Signal MA'].unique().size, df1['Old Signal MA'].size, df2['New Signal MA'].unique().size, df2['New Signal MA'].size


In [0]:
df2['New Signal MA'].where(df2['New Signal MA'].duplicated())

In [0]:
uploaded.keys()

dict_keys(['OLD Signal Number Lookup.xlsx'])